In [10]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
from Data_cleaning import *
from helpers import *



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("data\dataset\dataset")


In [58]:
# 1️⃣ Split data manually
x_tr = x_train[:50000]
y_tr = y_train[:50000]
x_tr, keep_mask = remove_nan_features(x_tr)
x_te = x_train[50000:70000, keep_mask]
y_te = y_train[50000:70000]
x_tr = x_tr[:,:20]
x_te = x_te[:,:20]

# 2️⃣ Compute column means from TRAINING set only
col_means = np.nanmean(x_tr, axis=0)

# 3️⃣ Impute missing values using TRAINING means
inds_tr = np.where(np.isnan(x_tr))
x_tr[inds_tr] = np.take(col_means, inds_tr[1])

inds_te = np.where(np.isnan(x_te))
x_te[inds_te] = np.take(col_means, inds_te[1])

x_tr, means, stds = standardize_features(x_tr)
x_te = (x_te - means) / stds



In [59]:
# 1️⃣ Convert labels to {0, 1}
y_tr = (y_tr + 1) / 2
y_te = (y_te + 1) / 2

# 2️⃣ Balance training data (undersampling)
X_tr_bal, y_tr_bal = balance_data(x_tr, y_tr_bal_input, method="oversample", random_state=42)

# 4️⃣ Check new balance
unique, counts = np.unique(y_tr_bal, return_counts=True)
print("Balanced training set:", dict(zip(unique, counts)))

Balanced training set: {np.int64(0): np.int64(45671), np.int64(1): np.int64(45671)}


In [63]:
print(y_tr_bal.shape, X_tr_bal.shape)

(91342,) (91342, 20)


In [62]:
loss, w = logistic_regression_penalized_gradient_descent_demo(y_tr_bal, X_tr_bal, 10000, 0.1, 0.0005)
print(loss, w)

Iteration     0, loss = 0.693147
Iteration   100, loss = 0.557844
Iteration   200, loss = 0.554099
Iteration   300, loss = 0.553654
Iteration   400, loss = 0.553553
Iteration   500, loss = 0.553502
Iteration   600, loss = 0.553462
Iteration   700, loss = 0.553428
Iteration   800, loss = 0.553397
Iteration   900, loss = 0.553369
Iteration  1000, loss = 0.553343
Iteration  1100, loss = 0.553319
Iteration  1200, loss = 0.553298
Iteration  1300, loss = 0.553278
Iteration  1400, loss = 0.553260
Iteration  1500, loss = 0.553243
Iteration  1600, loss = 0.553227
Iteration  1700, loss = 0.553213
Iteration  1800, loss = 0.553200
Iteration  1900, loss = 0.553188
Iteration  2000, loss = 0.553177
Iteration  2100, loss = 0.553167
Iteration  2200, loss = 0.553157
Iteration  2300, loss = 0.553149
Iteration  2400, loss = 0.553141
Iteration  2500, loss = 0.553133
Iteration  2600, loss = 0.553126
Iteration  2700, loss = 0.553120
Iteration  2800, loss = 0.553114
Iteration  2900, loss = 0.553108
Iteration 

In [64]:
print(np.sqrt(w.T @ w))

1.2026664287377935


In [65]:
def predict_labels_logistic(x, w, threshold=0.5, return_original_labels=True):

    # add bias term
    tx = np.c_[np.ones(x.shape[0]), x]
    
    # compute probabilities
    probs = sigmoid(tx @ w)
    
    # threshold
    preds = (probs >= threshold).astype(int)
    
    # convert to {-1, 1} if desired
    if return_original_labels:
        preds = 2 * preds - 1
    
    return preds

In [66]:

y_pred = predict_labels_logistic(x_te, w, return_original_labels=False)
n_errors = np.sum(np.abs(y_te - y_pred))
accuracy = np.mean(y_te == y_pred)

print("Misclassified samples:", n_errors)
print(f"Accuracy: {accuracy:.3f}")
print(np.sum(y_pred))


Misclassified samples: 5818.0
Accuracy: 0.709
6664


In [68]:
print(np.sum(y_te))

1772.0


In [70]:
print(np.sum(np.abs(y_pred - y_te)))

5818.0
